In [191]:
%load_ext autoreload
%autoreload 2
from monai.transforms import LoadImage, CenterSpatialCrop, Compose, SpatialPad
from monai.data import DataLoader, Dataset
import os
from utils.plotting import slice_comparison
import torch
from monai.visualize.utils import blend_images

idx = 5
models = ["UMambaBot","UMambaBotMTL", "SwinUNETR", "SwinUNETRMTL"]
pred_paths = [f"predictions/{model}/{case_id[idx]}/t2_pred.nii.gz" for model in models]

case_id = [name for name in os.listdir("predictions/UMambaBotMTL/")]

img_path = f"predictions/UMambaBotMTL/{case_id[idx]}/t2_img.nii.gz"
label_path = f"predictions/UMambaBotMTL/{case_id[idx]}/adc_tumor_reader1_gt.nii.gz"
# label_path = f"predictions/UMambaBotMTL/{case_id[idx]}/empty_gt.nii.gz"

# pred_path = f"predictions/{model}/{case_id[idx]}/t2/t2_trans.nii.gz"
# img_path = f"/work/michaesl/prostate/advanced_computing_2024/data/prostate158/test/{case_id[idx]}/t2.nii.gz"
# label_path = f"/work/michaesl/prostate/advanced_computing_2024/data/prostate158/test/{case_id[idx]}/adc_tumor_reader1.nii.gz"


trans = Compose(
    [LoadImage(ensure_channel_first=True),
     CenterSpatialCrop(roi_size=(224,224,20)),
     SpatialPad(spatial_size=(224,224,20))
     ]
)

preds = [trans(pred_path) for pred_path in pred_paths]
img = trans(img_path)
label = trans(label_path)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [192]:
def one_hot_to_label(one_hot_tensor, background_value=-1):
    """Converts a one-hot encoded tensor to a tensor of label values.

    Args:
        one_hot_tensor: A PyTorch tensor with shape [C, H, W, D].
        background_value: The value to use for background pixels 
                          where no channel is active (default: -1).

    Returns:
        A PyTorch tensor with shape [H, W, D] containing label values, 
        including the value for channel 0.
    """
    label_tensor = torch.argmax(one_hot_tensor, dim=0)
    # Create a mask for background pixels (where no channel is 1)
    background_mask = torch.all(one_hot_tensor == 0, dim=0)
    # Set background pixels to the specified value
    label_tensor[background_mask] = background_value
    return label_tensor[None]

In [193]:

titles = ["U-MambaBot","U-MambaBot MTL", "Swin UNETR", "Swin UNETR MTL", "Ground Truth"]
labels = [*preds, label]
labels = [one_hot_to_label(x) for x in labels]

slice_comparison(img[0][None], labels, titles)
# slice_comparison(img[0][None], [pred[0][None], label[0][None]], ["pred", "label"])

interactive(children=(IntSlider(value=0, description='Slice Index:', max=19), Output()), _dom_classes=('widget…

In [194]:

# labels = torch.argmax(pred, dim=0)

# volume_slice_plotter(labels, one_hot_to_label(pred))